---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [3]:
def graph_identification():
    
    df = pd.DataFrame(list(
        map(
            lambda x : (len(nx.center(x)) / len(x.nodes()) *100, len(nx.periphery(x)) / len(x.nodes())*100,
                        nx.average_shortest_path_length(x), nx.average_clustering(x)), P1_Graphs)),
                 columns = ['% nodes in center', '% nodes in periphery' , 'Average Shortest Path Length', 'Average clustering'])

    PA = list(df[
        abs(df['% nodes in center'] - df['% nodes in periphery']) > 
        (abs(df['% nodes in center'] - df['% nodes in periphery'])).mean()].index)
    PA = list(zip(PA, ['PA']*len(PA)))


    SW = df[
        abs(df['% nodes in center'] - df['% nodes in periphery']) < 
        (abs(df['% nodes in center'] - df['% nodes in periphery'])).mean()]

    SW_L = list(SW[SW['Average clustering'] > SW['Average clustering'].mean()].index)
    SW_L = list(zip(SW_L, ['SW_L']*len(SW_L)))

    SW_H = list(SW[SW['Average clustering'] < SW['Average clustering'].mean()].index)
    SW_H = list(zip(SW_H, ['SW_H']*len(SW_H)))

    
    return list(map(lambda x : x[1], sorted(PA + SW_L + SW_H)))
  
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [4]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [5]:
def salary_predictions():
    
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    
    dc = nx.degree_centrality(G)
    cc = nx.closeness_centrality(G, normalized = True)
    btc = nx.betweenness_centrality(G, normalized = True, endpoints = False)
    hub, auth = nx.hits(G)

    df = pd.DataFrame([n[1] for n in G.nodes(data = True)])
    for ind in range(len(df)):
        df.loc[ind, 'd'] = dc[ind]
        df.loc[ind, '$C_{close}$'] = cc[ind]
        df.loc[ind, 'C_{btw}$'] = btc[ind]
        df.loc[ind, 'hub'] = hub[ind]
        df.loc[ind, 'auth'] = auth[ind]

    X_prediction = df[df['ManagementSalary'].isnull()]
    df_training = df[df['ManagementSalary'].notnull()]
    
    features = [col for col in df.columns if col != 'ManagementSalary']
    X = df_training[features]
    y = df_training['ManagementSalary']
    X_prediction = X_prediction[features]

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    clf = RandomForestClassifier().fit(X_train, y_train)

    y_prediction = pd.Series(clf.predict_proba(X_prediction)[:, 1], 
              index = X_prediction.index)

    return y_prediction

salary_predictions()

1       0.1
2       0.9
5       1.0
8       0.3
14      0.3
18      0.2
27      0.6
30      0.7
31      0.7
34      0.0
37      0.2
40      0.2
45      0.0
54      0.2
55      0.0
60      0.6
62      1.0
65      1.0
77      0.0
79      0.6
97      0.0
101     0.0
103     0.2
108     0.4
113     0.1
122     0.0
141     0.6
142     1.0
144     0.1
145     0.1
       ... 
913     0.0
914     0.1
915     0.0
918     0.0
923     0.0
926     0.0
931     0.0
934     0.0
939     0.0
944     0.0
945     0.0
947     0.5
950     0.5
951     0.0
953     0.0
959     0.0
962     0.0
963     0.2
968     0.0
969     0.0
974     0.0
984     0.0
987     0.0
989     0.0
991     0.0
992     0.0
994     0.0
996     0.0
1000    0.2
1001    0.0
Length: 252, dtype: float64

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [6]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [7]:
def new_connections_predictions():
    
    cn_dep = {(x[0], x[1]) : x[2] for x in nx.cn_soundarajan_hopcroft(G, community = 'Department')}
    ra_dep = {(x[0], x[1]) : x[2] for x in nx.ra_index_soundarajan_hopcroft(G, community = 'Department')}

    cn_sal = {(x[0], x[1]) : x[2] for x in nx.cn_soundarajan_hopcroft(G, community = 'ManagementSalary')}
    ra_sal = {(x[0], x[1]) : x[2] for x in nx.ra_index_soundarajan_hopcroft(G, community = 'ManagementSalary')}


    pref = {(x[0], x[1]) : x[2] for x in nx.preferential_attachment(G)}
    jacc = {(x[0], x[1]) : x[2] for x in nx.jaccard_coefficient(G)}

    future_connections['Com Dept Common Neighbors'] = [cn_dep[ind] for ind in future_connections.index]
    future_connections['Com Dept Ressource Allocation'] = [ra_dep[ind] for ind in future_connections.index]

    future_connections['Com Sal Common Neighbors'] = [cn_sal[ind] for ind in future_connections.index]
    future_connections['Com Sal Ressource Allocation'] = [ra_sal[ind] for ind in future_connections.index]

    future_connections['Pref. Attachement'] = [pref[ind] for ind in future_connections.index]
    future_connections['Jaccard coef'] = [jacc[ind] for ind in future_connections.index]



    features = [col for col in future_connections.columns if col != 'Future Connection']

    X = future_connections[future_connections['Future Connection'].notnull()][features]
    y = future_connections[future_connections['Future Connection'].notnull()]['Future Connection']
    X_pred = future_connections[future_connections['Future Connection'].isnull()][features]



    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X, y)

    clf = RandomForestClassifier().fit(X_train, y_train)

    y_pred = pd.Series(clf.predict_proba(X_pred)[:, 1], index = X_pred.index)

    return y_pred

new_connections_predictions()

(107, 348)    0.000000
(542, 751)    0.015479
(20, 426)     0.200000
(50, 989)     0.014225
(942, 986)    0.008525
(324, 857)    0.016956
(13, 710)     0.000000
(19, 271)     0.100000
(319, 878)    0.013131
(659, 707)    0.013777
(49, 843)     0.010803
(208, 893)    0.007148
(377, 469)    0.000000
(405, 999)    0.000000
(129, 740)    0.000000
(292, 618)    0.000000
(239, 689)    0.006603
(359, 373)    0.098445
(53, 523)     0.200000
(276, 984)    0.007846
(202, 997)    0.009752
(604, 619)    0.000000
(270, 911)    0.018723
(261, 481)    0.050000
(200, 450)    1.000000
(213, 634)    0.018632
(644, 735)    0.000000
(346, 553)    0.000000
(521, 738)    0.023705
(422, 953)    0.000000
                ...   
(672, 848)    0.018723
(28, 127)     1.000000
(202, 661)    0.010732
(54, 195)     1.000000
(295, 864)    0.005100
(814, 936)    0.015281
(839, 874)    0.008525
(139, 843)    0.009524
(461, 544)    0.015133
(68, 487)     0.000000
(622, 932)    0.012799
(504, 936)    0.000000
(479, 528) 